In [1]:
import sys
import os

%load_ext autoreload

In [2]:
sys.path.append(os.path.abspath('common/src'))
sys.path.append(os.path.abspath('v1_0_0/src'))
sys.path.append(os.path.abspath('v1_0_1/src'))

In [3]:
import create_rawdf
import scraping
import create_prediction_population
import preprocessing
# import prediction
from feature_engineering_prediction import PredictionFeatureCreator
from feature_engineering import FeatureCreator
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
import re
import pandas as pd

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%autoreload

In [ ]:
kaisai_date_list = scraping.scrape_kaisai_date(from_="2022-01",to_="2022-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
kaisai_date_list

['20220105',
 '20220108',
 '20220109',
 '20220110',
 '20220115',
 '20220116',
 '20220122',
 '20220123',
 '20220129',
 '20220130',
 '20220205',
 '20220206',
 '20220212',
 '20220213',
 '20220219',
 '20220220',
 '20220226',
 '20220227',
 '20220305',
 '20220306',
 '20220312',
 '20220313',
 '20220319',
 '20220320',
 '20220321',
 '20220326',
 '20220327',
 '20220402',
 '20220403',
 '20220409',
 '20220410',
 '20220416',
 '20220417',
 '20220423',
 '20220424',
 '20220430',
 '20220501',
 '20220507',
 '20220508',
 '20220514',
 '20220515',
 '20220521',
 '20220522',
 '20220528',
 '20220529',
 '20220604',
 '20220605',
 '20220611',
 '20220612',
 '20220618',
 '20220619',
 '20220625',
 '20220626',
 '20220702',
 '20220703',
 '20220709',
 '20220710',
 '20220716',
 '20220717',
 '20220723',
 '20220724',
 '20220730',
 '20220731',
 '20220806',
 '20220807',
 '20220813',
 '20220814',
 '20220820',
 '20220821',
 '20220827',
 '20220828',
 '20220903',
 '20220904',
 '20220910',
 '20220911',
 '20220917',
 '20220918',

In [9]:
#kaisai_date_listを保存しておきたい場合実行
with open("kaisai_date_list.pickle","wb") as f:
    pickle.dump(kaisai_date_list,f)

In [ ]:
#kaisai_date_listをpickleから取り出したい場合実行
with open("kaisai_date_list.pickle","rb") as f:
    kaisai_date_list = pickle.load(f)

In [5]:
race_id_list = scraping.scrape_race_id_list(kaisai_date_list)

  0%|          | 0/108 [00:00<?, ?it/s]

In [7]:
race_id_list

[]

In [6]:
html_paths_race = scraping.scrape_html_race(race_id_list = race_id_list)

0it [00:00, ?it/s]

In [8]:
#html_paths_raceが消えた時用
html_paths_race = list(scraping.HTML_RACE_DIR.glob("*.bin"))
html_paths_race.size()

[WindowsPath('data/html/race/202301010101.bin'),
 WindowsPath('data/html/race/202301010102.bin'),
 WindowsPath('data/html/race/202301010103.bin'),
 WindowsPath('data/html/race/202301010104.bin'),
 WindowsPath('data/html/race/202301010105.bin'),
 WindowsPath('data/html/race/202301010106.bin'),
 WindowsPath('data/html/race/202301010107.bin'),
 WindowsPath('data/html/race/202301010108.bin'),
 WindowsPath('data/html/race/202301010109.bin'),
 WindowsPath('data/html/race/202301010110.bin'),
 WindowsPath('data/html/race/202301010111.bin'),
 WindowsPath('data/html/race/202301010112.bin'),
 WindowsPath('data/html/race/202301010201.bin'),
 WindowsPath('data/html/race/202301010202.bin'),
 WindowsPath('data/html/race/202301010203.bin'),
 WindowsPath('data/html/race/202301010204.bin'),
 WindowsPath('data/html/race/202301010205.bin'),
 WindowsPath('data/html/race/202301010206.bin'),
 WindowsPath('data/html/race/202301010207.bin'),
 WindowsPath('data/html/race/202301010208.bin'),
 WindowsPath('data/h

In [9]:
results = create_rawdf.create_results(html_path_list=html_paths_race)

  0%|          | 0/6910 [00:00<?, ?it/s]

table not found at 202401020809
table not found at 202403020501
table not found at 202403020504
table not found at 202403020808
table not found at 202403030104
table not found at 202403030202
table not found at 202403030203
table not found at 202403030208
table not found at 202403030411
table not found at 202403030504
table not found at 202403030505
table not found at 202403030603
table not found at 202403030604
table not found at 202403030606
table not found at 202404010604
table not found at 202404010605
table not found at 202404030601
table not found at 202404030701
table not found at 202404030704
table not found at 202404040101
table not found at 202404040204
table not found at 202404040304
table not found at 202404040406
table not found at 202404040602
table not found at 202404040701
table not found at 202405030211
table not found at 202405040108
table not found at 202405040203
table not found at 202405050412
table not found at 202405050701
table not found at 202405050702
table no

In [11]:
horse_id_list = results["horse_id"].unique()
len(horse_id_list)
html_paths_horse = scraping.scrape_html_horse(horse_id_list= horse_id_list,skip=True)

  0%|          | 0/16494 [00:00<?, ?it/s]

In [12]:
#馬の過去成績テーブル作成
horse_results = create_rawdf.create_horse_results(html_path_list=html_paths_horse)

  0%|          | 0/16494 [00:00<?, ?it/s]

In [13]:
race_info = create_rawdf.create_race_info(html_paths_race)

  0%|          | 0/6910 [00:00<?, ?it/s]

'NoneType' object has no attribute 'find' at 202405050701
'NoneType' object has no attribute 'find' at 202405050702
'NoneType' object has no attribute 'find' at 202405050703
'NoneType' object has no attribute 'find' at 202405050704
'NoneType' object has no attribute 'find' at 202405050705
'NoneType' object has no attribute 'find' at 202405050706
'NoneType' object has no attribute 'find' at 202405050707
'NoneType' object has no attribute 'find' at 202405050708
'NoneType' object has no attribute 'find' at 202405050709
'NoneType' object has no attribute 'find' at 202405050710
'NoneType' object has no attribute 'find' at 202405050711
'NoneType' object has no attribute 'find' at 202405050712
'NoneType' object has no attribute 'find' at 202405050801
'NoneType' object has no attribute 'find' at 202405050802
'NoneType' object has no attribute 'find' at 202405050803
'NoneType' object has no attribute 'find' at 202405050804
'NoneType' object has no attribute 'find' at 202405050805
'NoneType' obj

In [14]:
#レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()

In [15]:
#馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()

In [18]:
#レース情報テーブルの加工
race_info_preprocessed = preprocessing.process_race_info()

## モデル生成

In [8]:
fc = FeatureCreator()
features = fc.create_features()

In [6]:
INPUT_DIR = Path("data","02_features")
features = pd.read_csv(INPUT_DIR / "features.csv", sep = "\t")

In [ ]:
feature_cols = [
    'horse_id', 
    'jockey_id', 
    'trainer_id', 
    'sex',
    'age', 
    'weight', 
    'weight_diff', 
    # 'tansho_odds', 
    # 'popularity', 
    'impost',
    'wakuban', 
    'umaban', 
    'race_type', 
    'around', 
    'course_len', 
    'weather',
    'ground_state', 
    'race_class',
    'place',
    # 'rank_1race',
    # 'prize_1race',
    'rank_3races',
    'prize_3races',
    'rank_5races',
    'prize_5races',
    'rank_10races',
    'prize_10races',
    'rank_1000races',
    'prize_1000races'
]

In [9]:
#説明変数
X = features[feature_cols]

KeyError: "['rank_1race', 'prize_1race'] not in index"

In [ ]:
horse_results_preprocessed = preprocessing.process_horse_results(
    save_filename="horse_results_prediction.csv"
)

In [20]:
#特徴量作成
pfc = PredictionFeatureCreator()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\prediction_population\\population.csv'